In [ ]:
pip install flair

# Spam Text Classification using Flair

In [2]:
from flair.models import TextClassifier
from flair.data import Sentence
classifier = TextClassifier.load('en-sentiment')
sentence = Sentence('Flair is pretty neat!')
classifier.predict(sentence)
# print sentence with predicted labels
print('Sentence above is: ', sentence.labels)

c:\Users\abeda\anaconda3\envs\NC\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


2023-11-18 11:56:10,163 https://nlp.informatik.hu-berlin.de/resources/models/sentiment-curated-distilbert/sentiment-en-mix-distillbert_4.pt not found in cache, downloading to C:\Users\abeda\AppData\Local\Temp\tmpe01erp79


100%|██████████| 253M/253M [00:29<00:00, 9.02MB/s] 

2023-11-18 11:56:39,900 copying C:\Users\abeda\AppData\Local\Temp\tmpe01erp79 to cache at C:\Users\abeda\.flair\models\sentiment-en-mix-distillbert_4.pt


2023-11-18 11:56:41,430 removing temp file C:\Users\abeda\AppData\Local\Temp\tmpe01erp79


Sentence above is:  ['Sentence[5]: "Flair is pretty neat!"'/'POSITIVE' (0.9997)]


In [30]:
import pandas as pd
from sklearn.model_selection import train_test_split

data = pd.read_csv("./spam.csv", encoding='latin-1').sample(frac=1).drop_duplicates()
data = data[['v1', 'v2']].rename(columns={"v1":"label", "v2":"text"})
 
data['label'] = '__label__' + data['label'].astype(str)
train_data, temp_data = train_test_split(data, test_size=0.2, random_state=42)
test_data, dev_data = train_test_split(temp_data, test_size=0.5, random_state=42)
#
train_data.to_csv('train.csv', index=False, header=False)
test_data.to_csv('test.csv', index=False, header=False)
dev_data.to_csv('dev.csv', index=False, header=False)


In [39]:
from flair.data import Corpus
from flair.datasets import CSVClassificationCorpus
from flair.embeddings import WordEmbeddings, FlairEmbeddings, DocumentRNNEmbeddings
from flair.models import TextClassifier
from flair.trainers import ModelTrainer
from pathlib import Path
# load corpus by pointing to folder. Train, dev and test gets identified automatically
label_type ='binary'
column_name_map = {0: "label", 1: "text"}
corpus: Corpus = CSVClassificationCorpus(data_folder=Path('./'),
                                        column_name_map=column_name_map,
                                        skip_header=False,
                                        label_type ='binary')
word_embeddings = [WordEmbeddings('glove'), FlairEmbeddings('news-forward-fast'), FlairEmbeddings('news-backward-fast')]
document_embeddings = DocumentRNNEmbeddings(word_embeddings, hidden_size=512, reproject_words=True, reproject_words_dimension=256)

classifier = TextClassifier(document_embeddings, label_dictionary=corpus.make_label_dictionary(label_type=label_type),label_type=label_type, multi_label=False)
trainer = ModelTrainer(classifier, corpus)
trainer.train('./', max_epochs=10)

2023-11-18 14:42:08,128 Reading data from .
2023-11-18 14:42:08,129 Train: train.csv
2023-11-18 14:42:08,130 Dev: dev.csv
2023-11-18 14:42:08,132 Test: test.csv
2023-11-18 14:42:17,614 Computing label dictionary. Progress:


0it [00:00, ?it/s]
4135it [00:03, 1176.19it/s]

2023-11-18 14:42:21,248 Dictionary created for label 'binary' with 2 values: __label__ham (seen 3627 times), __label__spam (seen 508 times)


2023-11-18 14:42:21,351 ----------------------------------------------------------------------------------------------------
2023-11-18 14:42:21,360 Model: "TextClassifier(
  (embeddings): DocumentRNNEmbeddings(
    (embeddings): StackedEmbeddings(
      (list_embedding_0): WordEmbeddings(
        'glove'
        (embedding): Embedding(400001, 100)
      )
      (list_embedding_1): FlairEmbeddings(
        (lm): LanguageModel(
          (drop): Dropout(p=0.25, inplace=False)
          (encoder): Embedding(275, 100)
          (rnn): LSTM(100, 1024)
        )
      )
      (list_embedding_2): FlairEmbeddings(
        (lm): LanguageModel(
          (drop): Dropout(p=0.25, inplace=False)
          (encoder): Embedding(275, 100)
          (rnn): LSTM(100, 1024)
        )
      )
    )
    (word_reprojection_map): Linear(in_features=2148, out_features=256, bias=True)
    (rnn): GRU(256, 512, batch_first=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Linear(in_features=51

100%|██████████| 9/9 [01:40<00:00, 11.15s/it]

...

2023-11-18 17:18:09,751 DEV : loss 0.08240685611963272 - f1-score (micro avg)  0.9826


2023-11-18 17:18:10,539  - 1 epochs without improvement
2023-11-18 17:18:15,445 ----------------------------------------------------------------------------------------------------
2023-11-18 17:18:15,450 Loading model from best epoch ...


100%|██████████| 9/9 [01:26<00:00,  9.64s/it]

2023-11-18 17:19:45,080 
Results:
- F-score (micro) 0.9884
- F-score (macro) 0.9749
- Accuracy 0.9884

By class:
               precision    recall  f1-score   support

 __label__ham     0.9889    0.9978    0.9933       446
__label__spam     0.9851    0.9296    0.9565        71

     accuracy                         0.9884       517
    macro avg     0.9870    0.9637    0.9749       517
 weighted avg     0.9884    0.9884    0.9883       517

2023-11-18 17:19:45,081 ----------------------------------------------------------------------------------------------------


{'test_score': 0.988394584139265}

In [41]:
from flair.models import TextClassifier
from flair.data import Sentence
classifier = TextClassifier.load('./best-model.pt')
sentence = Sentence('Many thanks for your application to join the team...')
classifier.predict(sentence)
print(sentence.labels)

['Sentence[10]: "Many thanks for your application to join the team..."'/'__label__ham' (0.9988)]
